# Uso de LangChaing junto a  OPENAI, nos conectamos a Google a Wikipedia y usamos agentes.

## Instalamos, Cargamos librerias y nos logueamos con el modelo

In [ ]:
!pip install langchain langchain_openai langchain_experimental langchainhub google-api-python-client wikipedia -q #Instala en modo silencioso

In [131]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.utilities import GoogleSearchAPIWrapper
from chromadb.utils import embedding_functions
from langchain_core.tools import Tool
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from dotenv import load_dotenv, find_dotenv
import os

In [3]:
Api_Key = os.getenv('OPENAI_API_KEY')
#os.environ["OPENAI_API_KEY"] = "TuClave de OPENAI"
os.environ["OPENAI_API_KEY"] = Api_Key
llm = ChatOpenAI()

In [50]:
#Prueba del LLM sin agente
llm.invoke("Que sabes de la empresa VHNGROUP")

AIMessage(content='VHNGROUP es una empresa colombiana que se dedica al desarrollo de proyectos inmobiliarios en el país. La compañía cuenta con una amplia experiencia en el sector y ha llevado a cabo la construcción de diversos proyectos residenciales y comerciales en diferentes ciudades de Colombia. VHNGROUP se destaca por su compromiso con la calidad y la innovación en sus proyectos, así como por su enfoque en brindar soluciones integrales a sus clientes. Además, la empresa se ha posicionado como una de las principales compañías de desarrollo inmobiliario en el país, gracias a su sólida reputación y trayectoria en el mercado.', response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 18, 'total_tokens': 162}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-eef75c89-76f3-4905-953b-1794876318f3-0', usage_metadata={'input_tokens': 18, 'output_tokens': 144, 'total_tokens': 162})

## Creamos el modelo:

#### Primer Prompt, le asignamos personalidad

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente virtual de la empresa VHNGROUP"),
        ("user", "{input}")
    ]
)
chain = prompt | llm

In [24]:
#Prueba del LLM con personalidad pero sin agente
chain.invoke({"input": "Hola, que es VHNGROUP"})

AIMessage(content='¡Hola! VHNGROUP es una empresa que ofrece servicios de consultoría en tecnología, desarrollo de software, diseño web, marketing digital y soluciones empresariales. ¿En qué puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 31, 'total_tokens': 75}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8e9513c4-ec1c-4ab8-b7d9-f7791d09f27f-0', usage_metadata={'input_tokens': 31, 'output_tokens': 44, 'total_tokens': 75})

### Le agregamos memoria

In [41]:
# Hacemos que el modelo tenga memoria y guarde contexto.
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente virtual de la empresa VHNGROUP especializado en {ability}, puede responder preguntas sobre {ability}"),
        MessagesPlaceholder(variable_name="history"), #Inserta el historico de la conversación
        ("human", "{input}")
    ]
)
runnable = prompt | model #Ejecutamos el prompt y el modelo

In [43]:
store = {} #creamos el diccionario para guardar la memoria

# Creamos las sesiones
def get_session_history(session_id: str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory() #Separamos las session_id para organizar
    return store[session_id]
    
#Agregamos el input y la historia a la conversación
with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key='input',
    history_messages_key='history',
)

# Preguntamos y catalogamos la conversación
with_message_history.invoke(
    {"ability":"atencion al cliente", "input":"Que camaras de video vigilancia debo comprar para una casa?"},
    config={"configurable" : {"session_id": "abc123"}},
)1

Parent run 999c8764-bcd8-4876-b5c1-9b6600bd04a9 not found for run 74032884-e68a-4d15-aed2-5bc6ed2974d6. Treating as a root run.


AIMessage(content='Para elegir cámaras de videovigilancia para una casa, es importante considerar varios aspectos como el tamaño de la propiedad, las áreas que deseas cubrir, la calidad de imagen que necesitas, si requieres visión nocturna, si quieres cámaras fijas o con movimiento, entre otros. \n\nAlgunas recomendaciones generales a tener en cuenta al comprar cámaras de videovigilancia para una casa son:\n\n1. Resolución de la cámara: Se recomienda optar por cámaras con al menos 1080p de resolución para obtener imágenes claras y nítidas.\n\n2. Visión nocturna: Si necesitas monitorear áreas en la oscuridad, elige cámaras con visión nocturna para una vigilancia continua.\n\n3. Conectividad: Puedes optar por cámaras con conexión Wi-Fi para facilitar la instalación y la visualización remota a través de una aplicación móvil.\n\n4. Almacenamiento: Considera si deseas almacenar las grabaciones en la nube, en una tarjeta SD o en un disco duro externo.\n\n5. Ángulo de visión: Elige cámaras co

#### Preguntamos pero ahora recuperando la historia con el session_id

In [46]:
with_message_history.invoke(
    {"ability": "match", "input": "Que marcas me sugieres?"},
    config={"configurable":{"session_id": "abc123"}}
),

Parent run 8c4fe3c0-742a-44e8-8b42-e4f0c2832548 not found for run aa43ab44-6651-4075-ad54-65d96cb7ccf7. Treating as a root run.


(AIMessage(content='Algunas marcas reconocidas en el mercado de cámaras de videovigilancia para el hogar son:\n\n1. Arlo: Conocida por sus cámaras inalámbricas y con batería recargable, ofrecen una fácil instalación y conectividad a través de Wi-Fi.\n\n2. Nest: Ofrece cámaras de alta calidad con funciones avanzadas como reconocimiento facial y detección de actividad.\n\n3. Ring: Conocida por sus videoporteros con cámara integrada, también ofrece cámaras de seguridad para el hogar con opciones de conectividad y almacenamiento en la nube.\n\n4. Lorex: Especializada en sistemas de videovigilancia, ofrece una amplia gama de cámaras con diferentes características y opciones de almacenamiento.\n\n5. Hikvision: Marca reconocida por su calidad de imagen y tecnología avanzada en cámaras de videovigilancia.\n\nEstas son solo algunas de las marcas populares en el mercado. Te recomendaría investigar más sobre las características específicas de cada marca y modelo para elegir la que mejor se adapte

#### Preguntamos pero ahora con otro session_id en el cual no tenga historico

In [48]:
with_message_history.invoke(
    {"ability": "match", "input": "En Colombia, que marcas?"},
    config={"configurable":{"session_id": "abc124"}}
),
#La respuesta fue muy.... creativa. por se puede ver no guardo contexto con las interacciones previas

Parent run 1e6fda64-4220-48d1-a462-956453b2ab51 not found for run 1847c6e4-c085-4f59-930b-cd9535c27df9. Treating as a root run.


(AIMessage(content='VHNGROUP ofrece servicios de match en Colombia a marcas de diversos sectores, como moda, tecnología, alimentos, entre otros. Algunas marcas colombianas con las que hemos trabajado incluyen Juan Valdez, Éxito, Avianca, Grupo Éxito, entre otras. Si estás interesado en conocer más sobre nuestras colaboraciones con marcas en Colombia, no dudes en preguntar.', response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 42, 'total_tokens': 130}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a447ba3b-4bb2-4f5b-a3c2-a7548b9ee8cc-0', usage_metadata={'input_tokens': 42, 'output_tokens': 88, 'total_tokens': 130}),)

### Agregamos "Tools" o propiedades para conectarse a internet en este caso, Google

In [69]:
#para este caso usamos las siguientes librerias:
    #from langchain_community.utilities import GoogleSearchAPIWrapper
    #from langchain_core.tools import Tool
# y debemos instalar pip install google-api-python-client

Api_Key_Google = os.getenv('GOOGLE_API_KEY')
CSE_ID_Google = os.getenv('GOOGLE_CSE_ID')
os.environ["GOOGLE_CSE_ID"] = CSE_ID_Google
os.environ["GOOGLE_API_KEY"] = Api_Key_Google

search = GoogleSearchAPIWrapper(k=3) #Limitamos en numero de resultados

toolg = Tool(
    name="google_search",
    description="Busqueda de resultados en la pagina de VHNGROUP.",
    func=search.run,
)

In [124]:
#info del wrapper de google, usando google search, centrado en la empresa VHNGROUP
toolg.name, tool.description, tool.args

('google_search',
 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.',
 {'query': {'title': 'Query',
   'description': 'query to look up on wikipedia',
   'type': 'string'}})

In [125]:
# Le pedimos que nos busque camaras para mascotas
toolg.run("camara para mascotas")

'Que camara debemos usar para cuidar nuestro hogar o mascota? SKU:NHA-P610 DISPENSADOR DE COMIDA PARA MASCOTAS INTELIGENTE Modelo: MM105NXT55. ... Camara Ip 2Mp Bala H265+ Dwdr Ir30M Ip67 Semimetal $328,000. Detalles. ... de algún movimiento extraño o inesperado en su hogar. También podrás modificar los sensores de la cámara para que no recibir notificaciones por sus mascotas\xa0...'

#### Agregamos a Wikipedia

In [126]:
# debemos importar las siguientes librerias
#from langchain_community.tools import WikipediaQueryRun
#from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
toolk = WikipediaQueryRun(api_wrapper=api_wrapper)
toolk.name, toolk.description, toolk.args

('wikipedia',
 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.',
 {'query': {'title': 'Query',
   'description': 'query to look up on wikipedia',
   'type': 'string'}})

In [127]:
tools = [toolg, toolk] # Creamos una lista de herramientas, para pasarlas al modelo

### Agregamos agentes

In [128]:
# Debemos agregar instalar:
#pip install langchainhub
# e importar from langchain import hub
# instanciamos un nuevo modelo
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0) #La tempera ayuda a que el modelo sea lo mas preciso y alucine
prompt = hub.pull("hwchase17/openai-functions-agent") #Usamos un prompt ya creado
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [129]:
# Creamos el agente
#Debemos agreagar: from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt) #Creamos el agente, pasamos el modelo, las herramientas y el prompt que dara el comportamiento
#Creamos el ejecutor del agente, ya que los agentes se egecutan los agentes de manera ciclica
# Debemos importar from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools = tools, verbose=True)

#### Prueba de los agentes

In [134]:
# Prueba 1
agent_executor.invoke({"input":"hola!"})



> Entering new AgentExecutor chain...
¡Hola! ¿En qué puedo ayudarte hoy?

> Finished chain.


{'input': 'hola!', 'output': '¡Hola! ¿En qué puedo ayudarte hoy?'}

In [135]:
# Prueba 2
agent_executor.invoke({"input":"Que es VHNGROUP?"})
# Como podemos ver, el resultado mejora muchisimo.



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'VHNGROUP'}`


No good Wikipedia Search Result was found
Invoking: `google_search` with `VHNGROUP`


Soluciones integrales en seguridad electrónica, energía solar, vídeo vigilancia, alarmas, iluminación y mas. Conoce nuestros productos por sus categorias. · Sin categoría. 1 Product · Accesorios para Vehículos. 4 Products · Aire Acondicionado. 14 Products · Alarmas y ... +57 3203231397info@vhngroup.comLunes – Viernes 8 AM – 5 PMVHNGROUP. Información de Contacto: +57 3203231397info@vhngroup.comLunes – Viernes 8 AM – 5 PMVHNGROUP.VHNGROUP es una empresa que ofrece soluciones integrales en seguridad electrónica, energía solar, vídeo vigilancia, alarmas, iluminación y más. Puedes conocer sus productos en categorías como accesorios para vehículos, aire acondicionado, alarmas, entre otros. Puedes contactarlos a través del teléfono +57 3203231397 o enviando un correo a info@vhngroup.com. Su horario de atención es de

{'input': 'Que es VHNGROUP?',
 'output': 'VHNGROUP es una empresa que ofrece soluciones integrales en seguridad electrónica, energía solar, vídeo vigilancia, alarmas, iluminación y más. Puedes conocer sus productos en categorías como accesorios para vehículos, aire acondicionado, alarmas, entre otros. Puedes contactarlos a través del teléfono +57 3203231397 o enviando un correo a info@vhngroup.com. Su horario de atención es de lunes a viernes de 8 AM a 5 PM.'}

In [136]:
# Prueba 2
agent_executor.invoke({"input":"Que camara debo usar para mascotas?"})
# De igual manera, la respuesta mejora muchisimo mas.



> Entering new AgentExecutor chain...
Para elegir la cámara adecuada para mascotas, es importante considerar algunas características clave que pueden ser útiles para monitorear a tus mascotas cuando no estás en casa. Algunas características a tener en cuenta al elegir una cámara para mascotas son:

1. **Resolución de video**: Busca una cámara con una buena resolución de video para poder ver claramente a tus mascotas.

2. **Visión nocturna**: Es útil tener una cámara con visión nocturna para poder monitorear a tus mascotas incluso en la oscuridad.

3. **Audio bidireccional**: Una cámara con audio bidireccional te permite escuchar a tus mascotas y también hablar con ellas a través de la cámara.

4. **Detección de movimiento**: Algunas cámaras tienen la capacidad de detectar el movimiento de tus mascotas y enviarte notificaciones.

5. **Aplicación móvil**: Asegúrate de que la cámara tenga una aplicación móvil fácil de usar para que puedas acceder al video en tiempo real desde tu teléfon

{'input': 'Que camara debo usar para mascotas?',
 'output': 'Para elegir la cámara adecuada para mascotas, es importante considerar algunas características clave que pueden ser útiles para monitorear a tus mascotas cuando no estás en casa. Algunas características a tener en cuenta al elegir una cámara para mascotas son:\n\n1. **Resolución de video**: Busca una cámara con una buena resolución de video para poder ver claramente a tus mascotas.\n\n2. **Visión nocturna**: Es útil tener una cámara con visión nocturna para poder monitorear a tus mascotas incluso en la oscuridad.\n\n3. **Audio bidireccional**: Una cámara con audio bidireccional te permite escuchar a tus mascotas y también hablar con ellas a través de la cámara.\n\n4. **Detección de movimiento**: Algunas cámaras tienen la capacidad de detectar el movimiento de tus mascotas y enviarte notificaciones.\n\n5. **Aplicación móvil**: Asegúrate de que la cámara tenga una aplicación móvil fácil de usar para que puedas acceder al video 